In [1]:
from os import walk
from joblib import Parallel, delayed
import librosa
import python_speech_features
import numpy as np
import multiprocessing
import json
from deep_audio import Directory, Audio, JSON

In [2]:
num_cores = multiprocessing.cpu_count()

sampling_rate = 24000

library = 'psf' # librosa, psf

path = f'audios/{sampling_rate}'

f = Directory.filenames(path)

data = {
    "mapping": [],
    "mfcc": [],
    "labels": []
}

In [6]:
def process_directory(dir, index):
    signal, sr = Audio.read(f'{path}/{dir}', sr=sampling_rate, normalize=True)

    signal = np.array(signal)

    signal = signal[:len(signal) - len(signal) % (sr * 5)]

    segments = len(signal) // (sr * 5)

    m = {
        "mfcc": [],
        "labels": [index] * segments
    }

    for i in range(segments):
        start_sample = sr * i * 5
        finish_sample = start_sample + (sr * 5)

        sample = signal[start_sample:finish_sample]

        # sample = Audio.normalize(sample)

        # mfcc = Audio.mfcc(sample, rate=sr)

        n_mfcc = 13
        n_mels = 40
        n_fft = 512 
        hop_length = 160
        fmin = 0
        fmax = None

        if library == 'librosa':
            mfcc = librosa.feature.mfcc(y=sample, sr=sr, n_fft=n_fft,
                                        n_mfcc=n_mfcc, n_mels=n_mels,
                                        hop_length=hop_length,
                                        fmin=fmin, fmax=fmax, htk=False)
        elif library == 'psf':
            mfcc = python_speech_features.mfcc(signal=sample, samplerate=sr, winlen=n_fft / sr, winstep=hop_length / sr,
                                            numcep=n_mfcc, nfilt=n_mels, nfft=n_fft, lowfreq=fmin, highfreq=fmax,
                                            preemph=0.0, ceplifter=0, appendEnergy=False)

        # mfcc = librosa.feature.mfcc(sample, sr=sr, n_mfcc=13, hop_length=512, n_fft=2048, lifter=22)

        # mfcc = mfcc.T

        m['mfcc'].append(mfcc.tolist())

    print(f'{dir} -> segments: {segments}')
    return m

In [7]:
def object_mfcc_to_json(m):
    data['mapping'] = [file.replace('.wav', '') for file in f]

    for i in m:
        data['mfcc'].extend(i['mfcc'])
        data['labels'].extend(i['labels'])

    JSON.create_json_file(f'processed/mfcc/{library}/mfcc_{sampling_rate}.json', data)


In [8]:
if __name__ == '__main__':
    m = Parallel(n_jobs=num_cores, verbose=len(f), temp_folder='./tmp/')(
        delayed(process_directory)(i, j) for j, i in enumerate(f) if j <= 40)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 tasks      | elapsed:   34.3s
[Parallel(n_jobs=8)]: Done   2 tasks      | elapsed:   39.7s
[Parallel(n_jobs=8)]: Done   3 tasks      | elapsed:   40.6s
[Parallel(n_jobs=8)]: Done   4 tasks      | elapsed:   42.7s
[Parallel(n_jobs=8)]: Done   5 tasks      | elapsed:   43.5s
[Parallel(n_jobs=8)]: Done   6 tasks      | elapsed:   46.2s
[Parallel(n_jobs=8)]: Done   7 tasks      | elapsed:   48.3s
[Parallel(n_jobs=8)]: Done   8 tasks      | elapsed:   50.2s
[Parallel(n_jobs=8)]: Done   9 tasks      | elapsed:   56.1s
[Parallel(n_jobs=8)]: Done  10 tasks      | elapsed:   57.6s
[Parallel(n_jobs=8)]: Done  11 tasks      | elapsed:  1.0min
[Parallel(n_jobs=8)]: Done  12 tasks      | elapsed:  1.1min
[Parallel(n_jobs=8)]: Done  13 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  14 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done  15 tasks      | elapsed:  1.2min
[Parallel(

In [9]:
    object_mfcc_to_json(m)

In [10]:
len(data['labels'])

6572